In [4]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch import nn
from PIL import Image

In [6]:
# 加载预训练的VGG16模型
model = models.vgg16(pretrained=False)
model.load_state_dict()

e:\onedrive\桌面\毕业设计\算法\venv\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
e:\onedrive\桌面\毕业设计\算法\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
# 冻结模型的前面层的参数
for param in model.features.parameters():
    param.requires_grad = False

# 替换分类器部分以实现自定义的特征提取
# 假设我们要输出一个256维的特征向量
num_features = model.classifier[6].in_features
features = list(model.classifier.children())[:-1] # 移除最后一个全连接层
features.extend([nn.Linear(num_features, 256)]) # 添加自定义的全连接层
model.classifier = nn.Sequential(*features)

# 打印修改后的模型结构
print(model)

In [ ]:
# 图像预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG16要求的输入尺寸
    transforms.ToTensor(),  # 将图片转换为Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 归一化
])

# 加载图像（确保路径正确）
img = Image.open("path/to/your/image.jpg")
img_t = transform(img)
batch_t = torch.unsqueeze(img_t, 0)

In [ ]:
# 确保模型处于评估模式
model.eval()

# 提取特征
with torch.no_grad():
    features = model(batch_t)

print(features.shape)  # 输出特征的形状，确认是否为一维向量（例如，[1, 256]）